In [1]:
import os
import pandas as pd
from tqdm import tqdm

In [2]:
pd.isna(None)

True

In [3]:
import sys
sys.path.append('../code')

In [4]:
%load_ext autoreload
%autoreload 2

from libs import constants
from libs import io
from libs import scholar
from libs import helpers

In [5]:
aps_os_data_tar_gz = '../data/final_dataset.tar.gz'
aps_data_path = '../data/aps_20240130'


In [6]:
# 1. Read aps-oa data
df_pub = io.read_file_from_tar_gz_as_dataframe(aps_os_data_tar_gz, constants.APS_OA_PUBLICATIONS_FN) # id_author (oopenalex)
df_pub['ID'] = range(1, len(df_pub) + 1)
df_pub.rename(columns={'id_publication':'id_publication_oa'}, inplace=True)
print(f'\n df_pub: {df_pub.shape}  \n {df_pub.head(5)} \n')


 df_pub: (678874, 19)  
    id_publication_oa created_date                updated_date  \
0         2155512932   2016-06-24  2024-10-20T07:51:36.512074   
1         1990233377   2016-06-24  2024-11-02T19:41:40.382208   
2         2006493953   2016-06-24  2024-11-02T02:20:53.552587   
3         2035445126   2016-06-24  2024-10-07T20:59:41.180954   
4         1976015478   2016-06-24  2024-10-19T02:58:20.325244   

                                               title  \
0  Suppression of microbunching instability in th...   
1              Cryogenic permanent magnet undulators   
2  Induction voltage adders and the induction acc...   
3  Review of single bunch instabilities driven by...   
4  Three-dimensional time and frequency-domain th...   

                            doi  cited_by_count  has_full_text  \
0  10.1103/physrevstab.7.074401             221          False   
1  10.1103/physrevstab.7.050702             143          False   
2  10.1103/physrevstab.7.064801             108 

In [7]:
# publications mapping with aps
df_map = io.read_file_from_tar_gz_as_dataframe(aps_os_data_tar_gz, constants.APS_OA_PUBLICATIONS_MAPPING_FN)
df_map.rename(columns={'oa_id':'id_publication_oa', 'aps_id':'id_publication_aps'}, inplace=True)
print(f'\n df_map: {df_map.shape}  \n {df_map.head(5)} \n')


 df_map: (678874, 2)  
    id_publication_oa  id_publication_aps
0         2155512932                  25
1         1990233377                   7
2         2006493953                  17
3         2035445126                  32
4         1976015478                  20 



In [8]:
# join aps id
df_pub = df_pub.merge(df_map[['id_publication_oa','id_publication_aps']], on='id_publication_oa').reset_index()
print(f'\n df_pub: {df_pub.shape}  \n {df_pub.head(5)} \n')


 df_pub: (678874, 21)  
    index  id_publication_oa created_date                updated_date  \
0      0         2155512932   2016-06-24  2024-10-20T07:51:36.512074   
1      1         1990233377   2016-06-24  2024-11-02T19:41:40.382208   
2      2         2006493953   2016-06-24  2024-11-02T02:20:53.552587   
3      3         2035445126   2016-06-24  2024-10-07T20:59:41.180954   
4      4         1976015478   2016-06-24  2024-10-19T02:58:20.325244   

                                               title  \
0  Suppression of microbunching instability in th...   
1              Cryogenic permanent magnet undulators   
2  Induction voltage adders and the induction acc...   
3  Review of single bunch instabilities driven by...   
4  Three-dimensional time and frequency-domain th...   

                            doi  cited_by_count  has_full_text  \
0  10.1103/physrevstab.7.074401             221          False   
1  10.1103/physrevstab.7.050702             143          False   
2  10.

In [9]:
# 2. Read APS data

# topics: concept, area, discipline
df_topic_types = io.read_csv(io.path_join(aps_data_path, constants.APS_TOPIC_TYPES_FN)) # id_topic_type, name
df_topic_types.rename(columns={'name':'type_name'}, inplace=True)
print(f'\n df_topic_types: {df_topic_types.shape}  \n {df_topic_types.head(5)} \n')



 df_topic_types: (3, 2)  
    id_topic_type   type_name
0              0        area
1              1  discipline
2              2     concept 



In [10]:
# df_area = io.read_csv(io.path_join(aps_data_path, constants.APS_AREAS_FN)) # id_area, code, label
# df_area['new_id_topic'] = range(1, len(df_area) + 1)
# df_area.rename(columns={'id_area':'id_topic'}, inplace=True)
# print(f'\n df_area: {df_area.shape}  \n {df_area.head(5)} \n')

In [11]:
# df_concepts = io.read_csv(io.path_join(aps_data_path, constants.APS_CONCEPTS_FN)) # id_concept, id_facet, code, label
# df_concepts['new_id_topic'] = range(1, len(df_concepts) + 1)
# df_concepts.rename(columns={'id_concept':'id_topic'}, inplace=True)
# print(f'\n df_concepts: {df_concepts.shape}  \n {df_concepts.head(5)} \n')

In [12]:
# df_disciplines = io.read_csv(io.path_join(aps_data_path, constants.APS_DISCIPLINES_FN)) # id_discipline, code, label
# df_disciplines['new_id_topic'] = range(1, len(df_disciplines) + 1)
# df_disciplines.rename(columns={'id_discipline':'id_topic'}, inplace=True)
# print(f'\n df_concepts: {df_disciplines.shape}  \n {df_disciplines.head(5)} \n')

In [13]:
# publication-topic
df_publication_topics = io.read_csv(io.path_join(aps_data_path, constants.APS_PUBLICTION_TOPICS)) # id_publication, id_topic, id_topic_type, primary
df_publication_topics.rename(columns={'id_publication':'id_publication_aps'}, inplace=True)
df_publication_topics = df_publication_topics.merge(df_topic_types[['id_topic_type','type_name']], on='id_topic_type', how='left')
df_publication_topics.rename(columns={'id_publication':'id_publication_aps'}, inplace=True)
print(f'\n df_publication_topics: {df_publication_topics.shape}  \n {df_publication_topics.head(5)} \n')


 df_publication_topics: (597018, 5)  
    id_publication_aps  id_topic  id_topic_type primary   type_name
0                2096         0              1     NaN  discipline
1                2096         0              2     NaN     concept
2                2096         1              2    True     concept
3                2107         0              1     NaN  discipline
4                2107         2              2    True     concept 



In [14]:
# merge topics to oa_publications
df_pub = df_pub.merge(df_publication_topics[['id_publication_aps','id_topic','id_topic_type','type_name','primary']], on='id_publication_aps', how='left').drop(columns=['index'])
df_pub['id_publication_classification'] = range(1, len(df_pub) + 1)
df_pub.shape

(1174619, 25)

In [15]:
df_pub.head(10)

,id_publication_oa,created_date,updated_date,title,doi,cited_by_count,has_full_text,institutions_distinct_count,is_retracted,language,...,referenced_works_count,author_count_mismatch,id_journal,ID,id_publication_aps,id_topic,id_topic_type,type_name,primary,id_publication_classification
0,2155512932,2016-06-24,2024-10-20T07:51:36.512074,Suppression of microbunching instability in th...,10.1103/physrevstab.7.074401,221,False,3,False,en,...,10,author_count_matched,4,1,25,NaN,NaN,NaN,NaN,1
1,1990233377,2016-06-24,2024-11-02T19:41:40.382208,Cryogenic permanent magnet undulators,10.1103/physrevstab.7.050702,143,False,2,False,en,...,14,author_count_matched,6,2,7,NaN,NaN,NaN,NaN,2
2,2006493953,2016-06-24,2024-11-02T02:20:53.552587,Induction voltage adders and the induction acc...,10.1103/physrevstab.7.064801,108,False,0,False,en,...,14,author_count_matched,2,3,17,NaN,NaN,NaN,NaN,3
3,2035445126,2016-06-24,2024-10-07T20:59:41.180954,Review of single bunch instabilities driven by...,10.1103/physrevstab.7.124801,102,False,1,False,en,...,75,author_count_matched,8,4,32,NaN,NaN,NaN,NaN,4
4,1976015478,2016-06-24,2024-10-19T02:58:20.325244,Three-dimensional time and frequency-domain th...,10.1103/physrevstab.7.060703,78,False,1,False,en,...,26,author_count_matched,2,5,20,NaN,NaN,NaN,NaN,5
5,2035309657,2016-06-24,2024-10-16T21:16:35.007783,Design and characterization of a neutralized-t...,10.1103/physrevstab.7.083501,40,False,5,False,en,...,29,author_count_matched,11,6,42,NaN,NaN,NaN,NaN,6
6,2068760305,2016-06-24,2024-10-04T17:50:59.292684,Gas desorption and electron emission from 1 Me...,10.1103/physrevstab.7.093202,40,False,3,False,en,...,35,author_count_matched,1,7,22,NaN,NaN,NaN,NaN,7
7,2145666200,2016-06-24,2024-10-06T07:42:54.191032,Generation of angular-momentum-dominated elect...,10.1103/physrevstab.7.123501,40,False,7,False,en,...,10,author_count_matched,8,8,33,NaN,NaN,NaN,NaN,8
8,2141009273,2016-06-24,2024-09-25T08:15:52.896283,Experimental characterization of a space charg...,10.1103/physrevstab.7.080702,37,False,3,False,en,...,14,author_count_matched,11,9,18,NaN,NaN,NaN,NaN,9
9,1883308951,2016-06-24,2024-10-22T22:09:59.700565,Energy loss of a high-charge bunched electron ...,10.1103/physrevstab.7.061301,35,False,2,False,en,...,23,author_count_matched,2,10,36,NaN,NaN,NaN,NaN,10


In [16]:
df_pub.loc[0].id_publication_classification

1

In [17]:
df_pub.query("type_name=='area'")

,id_publication_oa,created_date,updated_date,title,doi,cited_by_count,has_full_text,institutions_distinct_count,is_retracted,language,...,referenced_works_count,author_count_mismatch,id_journal,ID,id_publication_aps,id_topic,id_topic_type,type_name,primary,id_publication_classification
198497,2771630417,2017-12-22,2024-11-05T19:21:11.277874,Unbiased All-Optical Random-Number Generator,10.1103/physrevx.7.041050,10,False,3,False,en,...,54,author_count_matched,328,105612,5949,0.0,0.0,area,NaN,198498
208273,3112167980,2020-12-21,2024-11-05T22:28:57.556908,Quantum Entanglement Growth under Random Unita...,10.1103/physrevx.7.031016,684,False,3,False,en,...,102,author_count_matched,331,111192,5972,3.0,0.0,area,NaN,208274
208274,3112167980,2020-12-21,2024-11-05T22:28:57.556908,Quantum Entanglement Growth under Random Unita...,10.1103/physrevx.7.031016,684,False,3,False,en,...,102,author_count_matched,331,111192,5972,1.0,0.0,area,NaN,208275
208275,3112167980,2020-12-21,2024-11-05T22:28:57.556908,Quantum Entanglement Growth under Random Unita...,10.1103/physrevx.7.031016,684,False,3,False,en,...,102,author_count_matched,331,111192,5972,11.0,0.0,area,NaN,208276
208292,3105808506,2020-11-23,2024-11-06T04:22:50.684882,Search for Axionlike Dark Matter through Nucle...,10.1103/physrevx.7.041034,196,False,23,False,en,...,80,author_count_matched,328,111193,5990,33.0,0.0,area,NaN,208293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667509,2987270616,2019-11-22,2024-09-27T00:29:44.241552,Power-law entanglement growth from typical pro...,10.1103/physrevresearch.1.033067,19,False,1,False,en,...,98,author_count_matched,2328,351514,205152,12.0,0.0,area,NaN,667510
667510,2987270616,2019-11-22,2024-09-27T00:29:44.241552,Power-law entanglement growth from typical pro...,10.1103/physrevresearch.1.033067,19,False,1,False,en,...,98,author_count_matched,2328,351514,205152,1.0,0.0,area,NaN,667511
667532,3103822756,2020-11-23,2024-09-19T16:51:10.475118,Critical scaling for yield is independent of d...,10.1103/physrevresearch.1.012002,10,False,1,False,en,...,50,author_count_matched,2330,351531,205151,23.0,0.0,area,NaN,667533
667533,3103822756,2020-11-23,2024-09-19T16:51:10.475118,Critical scaling for yield is independent of d...,10.1103/physrevresearch.1.012002,10,False,1,False,en,...,50,author_count_matched,2330,351531,205151,10.0,0.0,area,NaN,667534


In [18]:
df_pub.query("type_name=='concept'")

,id_publication_oa,created_date,updated_date,title,doi,cited_by_count,has_full_text,institutions_distinct_count,is_retracted,language,...,referenced_works_count,author_count_mismatch,id_journal,ID,id_publication_aps,id_topic,id_topic_type,type_name,primary,id_publication_classification
101,3103781159,2020-11-23,2024-11-07T16:48:03.304583,Direct longitudinal laser acceleration of elec...,10.1103/physrevaccelbeams.19.021303,96,False,7,False,en,...,33,author_count_matched,274,101,4954,724.0,2.0,concept,True,102
103,2259155355,2016-06-24,2024-09-30T22:05:11.571201,Comparison of the conditioning of high gradien...,10.1103/physrevaccelbeams.19.032001,54,False,1,False,en,...,4,author_count_matched,271,102,4965,648.0,2.0,concept,True,104
104,2259155355,2016-06-24,2024-09-30T22:05:11.571201,Comparison of the conditioning of high gradien...,10.1103/physrevaccelbeams.19.032001,54,False,1,False,en,...,4,author_count_matched,271,102,4965,639.0,2.0,concept,NaN,105
106,2472693295,2016-07-22,2024-10-19T21:50:37.026390,"Channeling, volume reflection, and volume capt...",10.1103/physrevaccelbeams.19.071001,43,False,1,False,en,...,31,author_count_matched,275,103,4966,653.0,2.0,concept,True,107
107,2472693295,2016-07-22,2024-10-19T21:50:37.026390,"Channeling, volume reflection, and volume capt...",10.1103/physrevaccelbeams.19.071001,43,False,1,False,en,...,31,author_count_matched,275,103,4966,729.0,2.0,concept,NaN,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174614,2769826572,2017-12-04,2024-09-25T03:42:22.061393,Publisher's Note: Förster-type triplet-polaron...,10.1103/physrevb.96.179904,0,False,0,False,en,...,0,author_count_matched,6700,678874,499949,451.0,2.0,concept,NaN,1174615
1174615,2769826572,2017-12-04,2024-09-25T03:42:22.061393,Publisher's Note: Förster-type triplet-polaron...,10.1103/physrevb.96.179904,0,False,0,False,en,...,0,author_count_matched,6700,678874,499949,141.0,2.0,concept,NaN,1174616
1174616,2769826572,2017-12-04,2024-09-25T03:42:22.061393,Publisher's Note: Förster-type triplet-polaron...,10.1103/physrevb.96.179904,0,False,0,False,en,...,0,author_count_matched,6700,678874,499949,739.0,2.0,concept,NaN,1174617
1174617,2769826572,2017-12-04,2024-09-25T03:42:22.061393,Publisher's Note: Förster-type triplet-polaron...,10.1103/physrevb.96.179904,0,False,0,False,en,...,0,author_count_matched,6700,678874,499949,1595.0,2.0,concept,True,1174618


In [19]:
df_pub.query("type_name=='discipline'")

,id_publication_oa,created_date,updated_date,title,doi,cited_by_count,has_full_text,institutions_distinct_count,is_retracted,language,...,referenced_works_count,author_count_mismatch,id_journal,ID,id_publication_aps,id_topic,id_topic_type,type_name,primary,id_publication_classification
100,3103781159,2020-11-23,2024-11-07T16:48:03.304583,Direct longitudinal laser acceleration of elec...,10.1103/physrevaccelbeams.19.021303,96,False,7,False,en,...,33,author_count_matched,274,101,4954,0.0,1.0,discipline,NaN,101
102,2259155355,2016-06-24,2024-09-30T22:05:11.571201,Comparison of the conditioning of high gradien...,10.1103/physrevaccelbeams.19.032001,54,False,1,False,en,...,4,author_count_matched,271,102,4965,0.0,1.0,discipline,NaN,103
105,2472693295,2016-07-22,2024-10-19T21:50:37.026390,"Channeling, volume reflection, and volume capt...",10.1103/physrevaccelbeams.19.071001,43,False,1,False,en,...,31,author_count_matched,275,103,4966,0.0,1.0,discipline,NaN,106
108,2545246800,2016-11-04,2024-10-28T02:31:23.698588,Beam shaping to improve the free-electron lase...,10.1103/physrevaccelbeams.19.100703,36,False,1,False,en,...,25,author_count_matched,279,104,4971,0.0,1.0,discipline,NaN,109
110,2398720845,2016-06-24,2024-09-30T16:39:46.725830,Sub-fs electron bunch generation with sub-10-f...,10.1103/physrevaccelbeams.19.054401,34,False,2,False,en,...,33,author_count_matched,278,105,4976,0.0,1.0,discipline,NaN,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174589,2764337368,2017-10-20,2024-09-19T06:01:29.635160,Comment on “Evidence of a first-order phase tr...,10.1103/physrevb.96.157102,3,False,3,False,en,...,8,author_count_matched,6701,678871,499936,6.0,1.0,discipline,NaN,1174590
1174594,2773110132,2017-12-22,2024-09-19T18:36:49.145245,Two-channel model for spin-relaxation noise,10.1103/physrevb.96.235439,2,False,2,False,en,...,19,author_count_matched,6704,678872,499917,6.0,1.0,discipline,NaN,1174595
1174608,2769181017,2017-12-04,2024-10-09T06:07:18.055794,Excitonic physics in a Dirac quantum dot,10.1103/physrevb.96.195434,1,False,2,False,en,...,30,author_count_matched,6699,678873,499947,6.0,1.0,discipline,NaN,1174609
1174612,2769826572,2017-12-04,2024-09-25T03:42:22.061393,Publisher's Note: Förster-type triplet-polaron...,10.1103/physrevb.96.179904,0,False,0,False,en,...,0,author_count_matched,6700,678874,499949,6.0,1.0,discipline,NaN,1174613


In [20]:
df_pub.query("type_name not in ['area','discipline','concept']")

,id_publication_oa,created_date,updated_date,title,doi,cited_by_count,has_full_text,institutions_distinct_count,is_retracted,language,...,referenced_works_count,author_count_mismatch,id_journal,ID,id_publication_aps,id_topic,id_topic_type,type_name,primary,id_publication_classification
0,2155512932,2016-06-24,2024-10-20T07:51:36.512074,Suppression of microbunching instability in th...,10.1103/physrevstab.7.074401,221,False,3,False,en,...,10,author_count_matched,4,1,25,NaN,NaN,NaN,NaN,1
1,1990233377,2016-06-24,2024-11-02T19:41:40.382208,Cryogenic permanent magnet undulators,10.1103/physrevstab.7.050702,143,False,2,False,en,...,14,author_count_matched,6,2,7,NaN,NaN,NaN,NaN,2
2,2006493953,2016-06-24,2024-11-02T02:20:53.552587,Induction voltage adders and the induction acc...,10.1103/physrevstab.7.064801,108,False,0,False,en,...,14,author_count_matched,2,3,17,NaN,NaN,NaN,NaN,3
3,2035445126,2016-06-24,2024-10-07T20:59:41.180954,Review of single bunch instabilities driven by...,10.1103/physrevstab.7.124801,102,False,1,False,en,...,75,author_count_matched,8,4,32,NaN,NaN,NaN,NaN,4
4,1976015478,2016-06-24,2024-10-19T02:58:20.325244,Three-dimensional time and frequency-domain th...,10.1103/physrevstab.7.060703,78,False,1,False,en,...,26,author_count_matched,2,5,20,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171281,3128867361,2021-02-15,2024-10-18T13:57:38.918231,Erratum: Effects of laser phase fluctuations o...,10.1103/physreva.40.7404,1,False,0,False,en,...,0,author_count_matched,988,678371,49920,NaN,NaN,NaN,NaN,1171282
1171282,1997712021,2016-06-24,2024-10-22T13:26:57.963194,Hyperfine structure of some excited states of<...,10.1103/physreva.40.2159,0,False,1,False,en,...,18,author_count_matched,979,678372,49948,NaN,NaN,NaN,NaN,1171283
1171283,2048242108,2016-06-24,2024-09-14T10:37:24.017660,Crystals with a twist,10.1103/physreva.40.4824,0,False,1,False,en,...,11,author_count_matched,980,678373,49924,NaN,NaN,NaN,NaN,1171284
1171284,2064715249,2016-06-24,2024-09-13T23:28:23.295947,Treating nodes and singularities in quantum si...,10.1103/physreva.40.1207,0,False,1,False,en,...,8,author_count_matched,981,678374,49912,NaN,NaN,NaN,NaN,1171285


In [21]:
df_pub.query("type_name in ['area','discipline','concept']").shape

(597022, 25)

In [22]:
def get_object(row):
    obj = {}
    obj['id_publication_classification'] = row.id_publication_classification
    obj['new_id_publication'] = row.ID
    obj['aps_id_publication'] = row.id_publication_aps
    obj['oa_id_publication'] = row.id_publication_oa # not needed but just in case
    obj['aps_id_topic'] = '' if helpers.is_none(row.id_topic) else int(row.id_topic)
    obj['aps_id_topic_type'] = '' if helpers.is_none(row.id_topic_type) else int(row.id_topic_type)
    obj['id_topic'] = '' if helpers.is_none(row.type_name) or helpers.is_none(row.id_topic) else f"{row.type_name[0]}_{int(row.id_topic):07}"
    obj['classification_type'] = '' if helpers.is_none(row.type_name) else row.type_name
    obj['primary'] = False if helpers.is_none(row.type_name) else row.primary in [constants.TRUE_STR, True]
    
    return obj

In [39]:
data = [get_object(row) for index, row in tqdm(df_pub.sample(20).iterrows(), total=len(df_pub), desc="Processing rows")]

Processing rows:   0%|          | 20/1174619 [00:00<04:09, 4700.03it/s]


In [42]:
id =0
id_aps = data[id]['aps_id_publication']
data[id]


{'id_publication_classification': 1062045,
 'new_id_publication': 617715,
 'aps_id_publication': 444896,
 'oa_id_publication': 2905014724,
 'aps_id_topic': 1494,
 'aps_id_topic_type': 2,
 'id_topic': 'c_0001494',
 'classification_type': 'concept',
 'primary': True}

In [43]:
df_pub.query("id_publication_aps==@id_aps")

,id_publication_oa,created_date,updated_date,title,doi,cited_by_count,has_full_text,institutions_distinct_count,is_retracted,language,...,referenced_works_count,author_count_mismatch,id_journal,ID,id_publication_aps,id_topic,id_topic_type,type_name,primary,id_publication_classification
1062041,2905014724,2018-12-22,2024-11-05T00:43:39.858710,Localization trajectory and Chern-Simons axion...,10.1103/physrevb.99.125414,5,False,7,False,en,...,58,author_count_matched,6148,617715,444896,6.0,1.0,discipline,NaN,1062042
1062042,2905014724,2018-12-22,2024-11-05T00:43:39.858710,Localization trajectory and Chern-Simons axion...,10.1103/physrevb.99.125414,5,False,7,False,en,...,58,author_count_matched,6148,617715,444896,1929.0,2.0,concept,NaN,1062043
1062043,2905014724,2018-12-22,2024-11-05T00:43:39.858710,Localization trajectory and Chern-Simons axion...,10.1103/physrevb.99.125414,5,False,7,False,en,...,58,author_count_matched,6148,617715,444896,1856.0,2.0,concept,NaN,1062044
1062044,2905014724,2018-12-22,2024-11-05T00:43:39.858710,Localization trajectory and Chern-Simons axion...,10.1103/physrevb.99.125414,5,False,7,False,en,...,58,author_count_matched,6148,617715,444896,1494.0,2.0,concept,True,1062045
1062045,2905014724,2018-12-22,2024-11-05T00:43:39.858710,Localization trajectory and Chern-Simons axion...,10.1103/physrevb.99.125414,5,False,7,False,en,...,58,author_count_matched,6148,617715,444896,1320.0,2.0,concept,NaN,1062046
1062046,2905014724,2018-12-22,2024-11-05T00:43:39.858710,Localization trajectory and Chern-Simons axion...,10.1103/physrevb.99.125414,5,False,7,False,en,...,58,author_count_matched,6148,617715,444896,1564.0,2.0,concept,NaN,1062047
1062047,2905014724,2018-12-22,2024-11-05T00:43:39.858710,Localization trajectory and Chern-Simons axion...,10.1103/physrevb.99.125414,5,False,7,False,en,...,58,author_count_matched,6148,617715,444896,1140.0,2.0,concept,NaN,1062048


In [44]:
[obj for obj in data if obj['aps_id_publication']==id_aps]

[{'id_publication_classification': 1062045,
  'new_id_publication': 617715,
  'aps_id_publication': 444896,
  'oa_id_publication': 2905014724,
  'aps_id_topic': 1494,
  'aps_id_topic_type': 2,
  'id_topic': 'c_0001494',
  'classification_type': 'concept',
  'primary': True}]

In [45]:
id_aps

444896